In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
# needed for notebook to find other directories
module_path = os.path.abspath(os.path.join('../mipnerf'))
if module_path not in sys.path:
    sys.path.append(module_path)
# needed for notebook to find other directories

In [ ]:
import collections
import flax
import jax
import jax.numpy as jnp
from jax import random
import gin
from absl import flags
import functools
from flax.training import checkpoints
from mipnerf.internal import utils
from mipnerf.internal import datasets
from mipnerf.internal import c2f_obb_dataset
from mipnerf.internal import obbpose_dataset
from mipnerf.internal import models
from mipnerf.internal import obbpose_model
from mipnerf.internal import d_models
from mipnerf.internal import d_models360

import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R

In [ ]:
class Config:
  """Configuration flags for everything."""
  dataset_loader: str = 'carla_dyn'  # The type of dataset loader to use.
  batching: str = 'timestep'  # Batch composition, [single_image, all_images].
  batch_size: int = 512  # The number of rays/pixels in each batch.
  factor: int = 4  # The downsample factor of images, 0 for no downsampling.
  spherify: bool = True  # Set to True for spherical 360 scenes.
  centering: bool = True  # this determines if poses are centered around zero or not
  render_path: bool = False  # If True, render a path. Used only by LLFF.
  llffhold: int = 11  # Use every Nth image for the test set. Used only by LLFF.
  timesteps: int = 5  # How many timesteps the current scene has (a bit of a crutch right now, integrate to dataset?)
  lr_init: float = 5e-4  # The initial learning rate.
  lr_final: float = 5e-6  # The final learning rate.
  lr_delay_steps: int = 2500  # The number of "warmup" learning steps.
  eps_delay_steps: int = 0  # The number of "warmup" learning steps.
  eps_init: int = 3  # Initial interval for near loss
  eps_final: int = 0.2  # Final interval for near loss
  l2_init: int = 1  # Initial value for l2reg for deformation model
  l2_final: int = 0  # Final value for l2reg for deformation model
  l2_delay_steps: int = 5000  # reduce l2reg after this many steps
  psreg_init: float = 10e5  # start value for pose regularization
  psreg_final: float = 10e-1  # end value for pose regularization
  psreg_delay_steps: int = 5000  # after this many steps, start decreasing psreg
  psreg_delay_mult: float = 1.0
  random_box: bool = True
  random_yaw: bool = False
  box_noise: float = 0.5
  yaw_noise: float = 5.  # rotational noise in degrees added to box yaw / heading angle
  c2f_steps: list = (5000, 10000, 15000)  # The number of steps after which rays of higher resolutions should be loaded
  lr_delay_mult: float = 0.01  # How much sever the "warmup" should be.
  grad_max_norm: float = 0.  # Gradient clipping magnitude, disabled if == 0.
  grad_max_val: float = 0.  # Gradient clipping value, disabled if == 0.
  max_steps: int = 200000  # The number of optimization steps.
  save_every: int = 50000  # The number of steps to save a checkpoint.
  print_every: int = 100  # The number of steps between reports to tensorboard.
  gc_every: int = 10000  # The number of steps between garbage collections.
  test_render_interval: int = 1  # The interval between images saved to disk.
  disable_multiscale_loss: bool = False  # If True, disable multiscale loss.
  randomized: bool = True  # Use randomized stratified sampling.
  near: float = 0.0  # Near plane distance.
  far: float = 200.  # Far plane distance.
  coarse_loss_mult: float = 0.1  # How much to downweight the coarse loss(es).
  weight_decay_mult: float = 0.  # The multiplier on weight decay.
  white_bkgd: bool = False  # If True, use white as the background (black o.w.).
  rand_bkgd: bool = False  # If True, use random color as background

In [ ]:
def _recenter_poses(poses):
    """Recenter poses according to the original NeRF code."""
    poses_ = poses.copy()
    bottom = np.reshape([0, 0, 0, 1.], [1, 4])
    c2w = _poses_avg(poses)
    c2w = np.concatenate([c2w[:3, :4], bottom], -2)
    bottom = np.tile(np.reshape(bottom, [1, 1, 4]), [poses.shape[0], 1, 1])
    poses = np.concatenate([poses[:, :3, :4], bottom], -2)
    poses = np.linalg.inv(c2w) @ poses
    poses_[:, :3, :4] = poses[:, :3, :4]
    poses = poses_
    return poses, c2w

def _poses_avg(poses):
    """Average poses according to the original NeRF code."""
    hwf = poses[0, :3, -1:]
    center = poses[:, :3, 3].mean(0)
    vec2 = _normalize(poses[:, :3, 2].sum(0))
    up = poses[:, :3, 1].sum(0)
    c2w = np.concatenate([_viewmatrix(vec2, up, center), hwf], 1)
    return c2w
    
def _normalize(x):
    """Normalization helper function."""
    return x / np.linalg.norm(x)
    
def _viewmatrix(z, up, pos):
    """Construct lookat view matrix."""
    vec2 = _normalize(z)
    vec1_avg = up
    vec0 = _normalize(np.cross(vec1_avg, vec2))
    vec1 = _normalize(np.cross(vec2, vec0))
    m = np.stack([vec0, vec1, vec2, pos], 1)
    return m

poses_arr = np.load('/home/tristram/data/carla/25_box/poses_bounds.npy')
poses = poses_arr[:, :-2].reshape([-1, 3, 5]).transpose([1, 2, 0])
bds = poses_arr[:, -2:].transpose([1, 0])
#poses = poses_arr[:, :-4].reshape([-1, 3, 5]).transpose([1, 2, 0])
#bds = poses_arr[:, -4:-2].transpose([1, 0])
#princip_point = poses_arr[:, -2:]

# Update poses according to downsampling.
poses[:2, 4, :] = np.array([320,480]).reshape([2, 1])
poses[2, 4, :] = poses[2, 4, :] * 1. / 4.0

poses = np.moveaxis(poses, -1, 0).astype(np.float32)
poses, c2w = _recenter_poses(poses)
poses[:, :3, 3] /= 5.0


box_dict = np.load('/home/tristram/data/carla/25_box/3D_boxes.npy', allow_pickle=True).item()
box_pose = []
box_ext = []
for key in box_dict:
    if 'center' in key:
        box_pose.append(box_dict[key])
    elif 'ext' in key:
        box_ext.append(box_dict[key])
box_pose = np.array(box_pose)
box_ext = np.array(box_ext)

box_pose = np.linalg.inv(c2w) @ box_pose
box_pose[:, :3, 3] /= 5.0 #* 2.0
box_ext /= 5.0 #* 2.0

yaw = R.from_matrix(np.linalg.inv(box_pose[:, :3, :3]))  # take inverse of rotation matrix to go from world to object
yaw = np.array(yaw.as_rotvec())   

obbpose = np.concatenate([box_pose[:, :3, 3], yaw], axis=-1)
rel_pose = {}
cars = []
bpose = [k for k in box_dict if 'center' in k]
for i, key in enumerate(bpose):
    if '1_' in key and 'center' in key:
        can_pose = box_pose[i]
        ts, car, _ = key.split('_')
        cars.append(int(car))
        rel_pose[ts + '_' + car + '_rel'] = np.eye(4)
        box_dict[key] = obbpose[i]
        box_dict[ts + '_' + car + '_ext'] = box_ext[i]
    else:
        ts, car, _ = key.split('_')
        cars.append(int(car))
        rel_pose[ts+'_'+car+'_rel'] = np.matmul(can_pose, np.linalg.inv(box_pose[i]))
        box_dict[key] = obbpose[i]
        box_dict[ts + '_' + car + '_ext'] = box_ext[i]
        
cars = np.unique(np.array(cars))

In [ ]:
BoxRays = collections.namedtuple(
    'BoxRays',
    ('origins', 'directions',
     'viewdirs', 'radii', 'lossmult',
     'near', 'far'))

config = Config()

dataset = obbpose_dataset.get_dataset('train', '/home/tristram/data/carla/25_box', config)

#example_batch = dataset.peek()
#example_batch['box'] = example_batch['box'][None, ...]

model, init_variables = obbpose_model.construct_mipnerf(
      random.PRNGKey(20200823), dataset.peek())
optimizer = flax.optim.Adam(config.lr_init).create(init_variables)
state = utils.TrainState(optimizer=optimizer)
del optimizer, init_variables

# Because this is only used for test set rendering, we disable randomization.
def render_eval_fn(variables, _, batch):
    return jax.lax.all_gather(
        model.apply(
                variables,
                random.PRNGKey(0),  # Unused.
                batch['rays'],
                batch['init'],
                batch['ext'],
                batch['ts'],
                randomized=False,
                white_bkgd=config.white_bkgd,
                rand_bkgd=False,
                alpha=batch['alpha']),
            axis_name='batch')

render_eval_pfn = jax.pmap(
    render_eval_fn,
    in_axes=(None, None, 0),  # Only distribute the data input.
    donate_argnums=(2,),
    axis_name='batch',
)

state = checkpoints.restore_checkpoint('/home/tristram/nerf_results/Carla_den_cntr02_8_128_1_128_mipopt_noise05/25_box', 
                                      target=state)

In [ ]:
opt_pose = np.array(state.optimizer.target['params']['box_centers'])

In [ ]:
init = []
for i in range(int(poses.shape[0]/5)):
    init.append(np.array(
        [np.concatenate(box_dict[str(i + 1) + '_' + str(c) + '_center'][:, None], axis=0) for c in
            cars]).reshape(-1, 6))
init = np.array(init).reshape(int(poses.shape[0]/5), -1, 6)

In [ ]:
mse_x = np.sum((opt_pose[:,:,0] - init[:,:,0])**2) / opt_pose.shape[0]
mse_y = np.sum((opt_pose[:,:,1] - init[:,:,1])**2) / opt_pose.shape[0]
mse_z = np.sum((opt_pose[:,:,2] - init[:,:,2])**2) / opt_pose.shape[0]

mse_angle = np.sum((opt_pose[:,:,3:] - init[:,:,3:])**2) / opt_pose.shape[0]
print('mse_angle_opt:', mse_angle*180/np.pi)
example_batch = dataset.peek()
mse_init = np.sum((example_batch['init'][0][:,:,:3] - init[:,:,:3])**2) / opt_pose.shape[0]
mse_init_x = np.sum((example_batch['init'][0][:,:,0] - init[:,:,0])**2) / opt_pose.shape[0]
mse_init_y = np.sum((example_batch['init'][0][:,:,1] - init[:,:,1])**2) / opt_pose.shape[0]
mse_init_z = np.sum((example_batch['init'][0][:,:,2] - init[:,:,2])**2) / opt_pose.shape[0]
mse_ogangle = np.sum((example_batch['init'][0][:,:,3:] - init[:,:,3:])**2) / opt_pose.shape[0]
print('mse_angle_init:', mse_ogangle*180/np.pi)
print('mse_init:', mse_init_x+mse_init_y + mse_init_z)
print('mse_opt:', (mse_x+mse_y+mse_z) / 2)
print('mse_x:', mse_x, mse_init_x)
print('mse_y:', mse_y, mse_init_y)
print('mse_z:', mse_z, mse_init_z)
for j in range(3):
    if j == 0:
        print('mse_x:')
    elif j == 1:
        print('mse_y')
    elif j == 2:
        print('mse_z')
    for i in range(opt_pose.shape[1]):
        print(np.sum(((opt_pose[:,i,j] - init[:,i,j])**2) / opt_pose.shape[0]))
print('mse_total_car:')
for i in range(opt_pose.shape[1]):
    print(np.sum(((opt_pose[:,i,:3] - init[:,i,:3])**2) / opt_pose.shape[0]))